In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import feature_manager as fma
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks, losses
import visualizer
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
import importlib
import tr_utils
import datetime

2023-02-02 23:17:07.696946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-02 23:17:07.697112: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-02 23:17:07.697125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-02 23:17:09.296522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-02 23:17:09.296587: I tensorflow/compiler/xla/stream_exe

In [ ]:
symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

fm = fma.FeatureManager(
    target_col="trade_signal"
)

fm.import_trading_data(
    symbol=symbol,
    trade_timeframe=trade_tf,
)

fm.import_granular_data(
    symbol=symbol,
    granular_timeframe=granular_tf
)

In [ ]:
fm.df["swing"] = (fm.df["High"]-fm.df["Low"])/fm.df["Close"]

In [ ]:
fm.df["swing"].describe()

In [ ]:
importlib.reload(fma)

In [ ]:
tp = 0.03
sl = 0.03
md = 12

fm.prepare_trade_forward_data(
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
)

In [ ]:
fm.build_features(lags=12)

In [ ]:
importlib.reload(dnn)

In [ ]:
DEFAULT_PRINT_METRICS = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

results_list = []
results = dnn.evaluate_classifier_k_folds(
    hu = 500,
    fm = fm,
    fold_number = 10,
    batch_size = 1024,
    set_class_weight = False,
    save_check_point = True,
    early_stop = True,
    rebalance = None,
    split_type="time_series_split",
    metrics=DEFAULT_PRINT_METRICS,
    set_initial_bias = True,
    dropout = True,
    dropout_rate = 0.3,
    shuffle_when_train = True,
    gpu = False,
    write_to_file = True
)
results_list.append(results)

In [ ]:
fm.df["gold_lag_1"].value_counts()

In [ ]:
metric_list = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

In [ ]:
results_list = []
results = dnn.evaluate_classifier(
    hu=100,
    fm=fm,
    laps=1,
    batch_size=20,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    shuffle_before_split=False,
    dropout=True,
    dropout_rate=0.3,
    gpu=False,
    metrics=metric_list,
    write_to_file=True
)
results_list.append(results)

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv("../out/evaluate.csv")

In [ ]:
importlib.reload(dnn)

In [ ]:
classifier = dnn.MultiDNNClassifer()

filename = "../logs/report/{}.txt".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
report_file = open(filename,'w')

dataset = classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    shuffle_before_split= False,
    categorical_label=True,
    rebalance="over",
    target_col="trade_signal",
    file = report_file
)

In [ ]:
initial_bias = tr_utils.init_imbalanced_bias(
    y_train=dataset[1]
)

classifier.configure(
    hu = 100, 
    dropout=True, 
    dropout_rate = 0.3,
    input_dim=len(fm.cols),
    output_bias=initial_bias,
    class_num=3
)

test_result = classifier.run(
    gpu = False,
    dataset = dataset,
    epochs = 500,
    shuffle_when_train = False,
    patience = 5,
    early_stop = True,
    save_check_point = True,
    set_class_weight = False,
    batch_size = 128,
    file = report_file
)

var_df = pd.DataFrame(data=classifier.viann_callback.varScores, index = fm.cols)
var_df.columns = ["var"]
var_df.sort_values(by=["var"],ascending=False)
